In [ ]:
import torch
from torch import nn
from torchvision import models
import timm


weights = "D:\\WorkFile\\Master\\Project\\Code\\Github\\finetuned_weights\\vit.pth"
checkpoint = torch.load(weights, map_location="cpu")
print(checkpoint.keys())

model = timm.create_model('vit_base_patch8_224', pretrained=True)

print(list(checkpoint.keys())[0].split(".")[0])

for k in list(checkpoint):
    tmp = "."
    checkpoint[tmp.join(k.split(".")[1:])] = checkpoint.pop(k)

print(checkpoint.keys())

model.load_state_dict(checkpoint)


In [3]:
import timm

model = timm.create_model('vit_base_patch8_224', pretrained=True)
model.blocks = model.blocks[:10]
model.head = nn.Linear(in_features=model.head.in_features, out_features=3)

print(model)

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(8, 8), stride=(8, 8))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop2): Dropout(p=0.0, inplace=False

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools

def draw_confusion_matrix(cm, classes, normalize=True, title='Confusion matrix', cmap=plt.cm.Blues):
    """
    - cm : 计算出的混淆矩阵的值
    - classes : 混淆矩阵中每一行每一列对应的列
    - normalize : True:显示百分比, False:显示个数
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("显示百分比：")
        np.set_printoptions(formatter={'float': '{: 0.2f}'.format})
        print(cm)
    else:
        print('显示具体数字：')
        print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    #  matplotlib版本问题，如果不加下面这行代码，则绘制的混淆矩阵上下只能显示一半，有的版本的matplotlib不需要下面的代码，分别试一下即可
    plt.ylim(len(classes) - 0.5, -0.5)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

class ClassifyMetric:
    # 这里的 labels 可以使用 None，这样就会让sklearn自己决定标签
    def __init__(self, numClass, labels=None):
        self.labels = labels
        self.numClass = numClass
        self.confusionMatrix = np.zeros((self.numClass,)*2)

    def genConfusionMatrix(self, y_true, y_pred):
        return confusion_matrix(y_true, y_pred, labels=self.labels)

    def addBatch(self, y_true, y_pred):
        assert np.array(y_true).shape == np.array(y_pred).shape
        self.confusionMatrix += self.genConfusionMatrix(y_true, y_pred)

    def reset(self):
        self.confusionMatrix = np.zeros((self.numClass, self.numClass))

    def accuracy(self):
        accuracy = np.diag(self.confusionMatrix).sum() / \
            self.confusionMatrix.sum()
        return accuracy

    def precision(self):
        precision = np.diag(self.confusionMatrix) / \
            self.confusionMatrix.sum(axis=0)
        return np.nan_to_num(precision)

    def recall(self):
        recall = np.diag(self.confusionMatrix) / \
            self.confusionMatrix.sum(axis=1)
        return recall

    def f1_score(self):
        precision = self.precision()
        recall = self.recall()
        f1_score = 2 * (precision*recall) / (precision+recall)
        return np.nan_to_num(f1_score)

metrics = ClassifyMetric(3, ["LP", "LX", "RT"])
label       = ['LP', 'LP', 'LP', 'LP', 'LP', 'LP', 'LP', 'LP', 'LP', 'LX', 'LX', 'LX', 'LX', 'LX', 'LX', 'RT', 'RT', 'RT', 'RT', 'RT', 'RT', 'RT', 'RT', 'RT', 'RT', 'RT', 'RT', 'RT', 'RT', 'RT', 'RT', 'RT', 'RT', 'RT']
prediction  = ['LP', 'LP', 'LP', 'LP', 'LP', 'LP', 'LP', 'LP', 'LP', 'LX', 'LX', 'LX', 'LX', 'LX', 'LX', 'LX', 'RT', 'RT', 'RT', 'LX', 'RT', 'RT', 'RT', 'RT', 'RT', 'RT', 'RT', 'RT', 'RT', 'RT', 'LX', 'RT', 'RT', 'LX']
metrics.addBatch(label, prediction)
matrix = metrics.genConfusionMatrix(label, prediction)
draw_confusion_matrix(matrix, ['LP', 'LX', 'RT'],)

accuracy = metrics.accuracy()
precision = metrics.precision()
recall = metrics.recall()
f1_score = metrics.f1_score()

print(40 * '*')
print('accuracy:')
print(accuracy)
print('precision:')
print(precision, end=", Average: ")
print(np.mean(precision))
print('recall:')
print(recall, end=", Average: ")
print(np.mean(recall))
print('f1_score:')
print(f1_score, end=", Average: ")
print(np.mean(f1_score))

In [4]:
import torch
from torchvision import models
import torch.nn as nn
from components.se_sep_transformer import SeSepViT, FuSepViT
from vit_pytorch.sep_vit import SepViT
from components.senet.se_resnet import se_resnet18, se_resnet20, se_resnet34, se_resnet50, se_resnet56, se_resnet101, se_resnet152
from components.fusion import fusion
import timm

from thop import profile

input = torch.randn(1, 3, 224, 224)

# model = SeSepViT(                                                               
#         num_classes = 1000,
#         dim = 32,            
#         dim_head = 32,   
#         heads = (4, 8, 16, 32),
#         depth = (1, 2, 14, 2),
#         window_size = 7,      
#         dropout = 0.1     
#     )

# model1 = FuSepViT(                                                               
#             num_classes = 1000,
#             dim = 32,           
#             dim_head = 32,          
#             heads = (1, 2, 4, 8),  
#             depth = (1, 2, 6, 2),  
#             window_size = 7,       
#             dropout = 0.1          
#         )
# print("Loading pretrained weights...")
# checkpoint = torch.load("..\\pretrained_weights\\sepvit_lite.pth", map_location="cpu")["state_dict"]
# if list(checkpoint.keys())[0].split(".")[0] == "module":
#     for k in list(checkpoint):
#         tmp = "."
#         checkpoint[tmp.join(k.split(".")[1:])] = checkpoint.pop(k)
# checkpoint.pop("mlp_head.1.weight")
# checkpoint.pop("mlp_head.1.bias")
# checkpoint.pop("mlp_head.2.weight")
# checkpoint.pop("mlp_head.2.bias")
# model1.load_state_dict(checkpoint)
# model2 = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
# layers = list(model2.children())[:-2]
# model2 = torch.nn.Sequential(*layers)
# model = fusion(model1, model2, 768, 3)
# model = models.resnet18()
model = timm.create_model('vit_base_patch8_224', pretrained=True)
model.blocks = model.blocks[:10]
model.head = nn.Linear(in_features=model.head.in_features, out_features=3)


flops, params = profile(model, (input,))
print("FLOPS(G):")
print(flops / (2 ** 30))
print("Params(MB)")
print(params / (2 ** 20))

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.normalization.LayerNorm'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
FLOPS(G):
51.90043759346008
Params(MB)
67.74023723602295


In [ ]:
import logging
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import RocCurveDisplay
from itertools import cycle

lg = logging.getLogger()
lg.setLevel("INFO") # 不想打印log时，把INFO替换成ERROR

iris = load_iris()
target_names = iris.target_names

lg.info("=" * 80)
lg.info("target_names:")
lg.info(target_names)

X, y = iris.data, iris.target

lg.info("=" * 80)
lg.info("X:")
lg.info(X[:3])
lg.info("X.shape:")
lg.info(X.shape)
lg.info("=" * 80)
lg.info("y:")
lg.info(y[:3])

y = iris.target_names[y]

lg.info("=" * 80)
lg.info(y[:3])

random_state = np.random.RandomState(0)
n_samples, n_features = X.shape
n_classes = len(np.unique(y))

lg.info("=" * 80)
lg.info("n_samples:")
lg.info(n_samples)
lg.info("=" * 80)
lg.info("n_features:")
lg.info(n_features)
lg.info("=" * 80)
lg.info("n_classes:")
lg.info(n_classes)

# 增长每个样本的特征向量，在每个特征向量后面拼接随机值
X = np.concatenate([X, random_state.randn(n_samples, 200 * n_features)], axis=1)

lg.info("=" * 80)
lg.info("X.shape after concatenate:")
lg.info(X.shape)

(X_train, X_test, y_train, y_test,) = train_test_split(X, y, test_size=0.5, stratify=y, random_state=0)

# 训练分类器
classifier = LogisticRegression()
y_score = classifier.fit(X_train, y_train).predict_proba(X_test)

lg.info("=" * 80)
lg.info("y_score:")
lg.info(y_score[:3])

# 制作one_hot标签
label_binarizer = LabelBinarizer().fit(y_train)
lg.info("=" * 80)
lg.info("y_test:")
lg.info(y_test[:3])
y_onehot_test = label_binarizer.transform(y_test)

# y_onehot_test = LabelBinarizer().fit_transform(y_test)

lg.info("y_onehot_test:")
lg.info(y_onehot_test[:3])
lg.info("y_onehot_test.shape:")
lg.info(y_onehot_test.shape)


fig, ax = plt.subplots(figsize=(6, 6))

colors = cycle(["aqua", "darkorange", "cornflowerblue"])
for class_id, color in zip(range(n_classes), colors):
    RocCurveDisplay.from_predictions(
        y_onehot_test[:, class_id],
        y_score[:, class_id],
        name=f"ROC curve for {target_names[class_id]}",
        color=color,
        ax=ax,
    )

plt.plot([0, 1], [0, 1], "k--", label="ROC curve for chance level (AUC = 0.5)")
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Extension of Receiver Operating Characteristic\nto One-vs-Rest multiclass")
plt.legend()
plt.show()

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 

x = np.arange(-10, 10, 0.001)
a = x[x > 0]
b = 0.1 * x[x < 0]
y = np.concatenate((b, a), axis=0)
plt.plot(x,y)
plt.suptitle(r'$y=\frac{1}{1+e^{-x}}$', fontsize=20)
plt.grid(color='gray')
plt.grid(linewidth='1')
plt.grid(linestyle='--')

plt.show()

In [ ]:
from itertools import cycle
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay
import numpy as np 

one_hot_target = np.array([[0, 1], [1, 0], [0, 1], [1, 0], [0, 1], [1, 0], [0, 1], [1, 0], [1, 0], [0, 1]])
confidence = np.array([[0.27, 0.73], [0.64, 0.36], [0.15, 0.85], [0.97, 0.03], [0.42, 0.58], [0.89, 0.11], [0.21, 0.79], [0.32, 0.68], [0.47, 0.53], [0.81, 0.19]])
idx_to_class = {0: '0', 1: '1'}

fig, ax = plt.subplots(figsize=(6, 6))
colors = cycle(["aqua", "darkorange"])

for class_id, color in zip(range(2), colors):
    RocCurveDisplay.from_predictions(
        one_hot_target[:, class_id],
        confidence[:, class_id],
        name=f"ROC curve for {idx_to_class[class_id]}",
        color=color,
        ax=ax,
    )

plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Extension of Receiver Operating Characteristic\nto One-vs-Rest multiclass")
plt.legend()
plt.show()

In [ ]:
from components.senet.se_resnet import se_resnet18
import torch
import torchvision.models as models
from components.se_sep_transformer import FuSepViT


x = torch.randn(16, 3, 224, 224)


model = se_resnet18()
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
layers = list(model.children())[:-2]
res = torch.nn.Sequential(*layers)
print(res(x).shape)


# model = FuSepViT(                                                               
#             num_classes = 1000,
#             dim = 32,           
#             dim_head = 32,          
#             heads = (1, 2, 4, 8),  
#             depth = (1, 2, 6, 2),  
#             window_size = 7,       
#             dropout = 0.1          
#         )
# print(model(x).shape)


In [ ]:
import torch
from vit_pytorch.sep_vit import SepViT


x = torch.randn(16, 3, 224, 224)

model = model = SepViT(                                                               
            num_classes = 1000,
            dim = 32,           
            dim_head = 32,          
            heads = (1, 2, 4, 8),  
            depth = (1, 2, 6, 2),  
            window_size = 7,       
            dropout = 0.1          
        )

checkpoint = torch.load("..\\pretrained_weights\\sepvit_lite.pth", map_location="cpu")["state_dict"]
if list(checkpoint.keys())[0].split(".")[0] == "module":
            for k in list(checkpoint):
                tmp = "."
                checkpoint[tmp.join(k.split(".")[1:])] = checkpoint.pop(k)
model.load_state_dict(checkpoint)
checkpoint.pop("mlp_head.1.weight")
checkpoint.pop("mlp_head.1.bias")
checkpoint.pop("mlp_head.2.weight")
checkpoint.pop("mlp_head.2.bias")
print(checkpoint.keys())